# Notebook 01 — Web Scraping Goodreads

In this notebook, we will collect data from a public Goodreads list using web scraping.  
We will extract the **title**, **author**, **rating**, and **link** of each book from the list  
*"Best Books Ever"* as a starting point for our dataset.

Additionally, we will extend the scraping process to capture complementary information such as  
**genre**, **price**, **number of reviews**, **published year**, and **book cover**, 
which will enrich the dataset for later analysis and recommendations.

## Step 1 — Imports & Setup

In this step, we import all the required libraries and project functions.
We also load the `config.yaml` file from the root directory to manage all
file paths dynamically (for raw, clean, and other data folders).

The `functions.py` module, located inside the `notebooks` folder, contains
reusable utilities for web scraping, API fetching, and dataset handling.

In [ ]:
# ============================================================
# Step 1 — Imports & Setup (safe simple version)
# ============================================================

import sys
from pathlib import Path
from time import sleep
import random
import importlib

sys.path.append("notebooks")

from functions import (
    load_config,
    ensure_directories,
    fetch_html,
    save_html,
    parse_html,
    extract_books_from_soup,
    save_dataset
)
import functions
importlib.reload(functions)

# --- Load config directly from root ---
config_path = Path("../config.yaml")
config = load_config(config_path)

# --- Ensure directories once ---
ensure_directories(config["paths"])

print("✅ All functions loaded successfully!")


## Step 2 — Define URL & Scrape Goodreads List

In this step, we will define the Goodreads list URL and send a request to fetch its HTML content.  
We will then parse the page using BeautifulSoup to extract the book data (title, author, rating, link, and other details like genre, price, and published year).  
Finally, we will save the raw HTML file in the `data/raw` folder for future reference.


In [ ]:
# ============================================================
# Step 2 — Define URL & Scrape Goodreads List
# ============================================================

from functions import fetch_html, save_html, parse_html
from pathlib import Path

# --- Define URL ---
url = "https://www.goodreads.com/list/show/1.Best_Books_Ever"

# --- Fetch & parse HTML ---
html_content = fetch_html(url)
soup = parse_html(html_content)

# --- Save raw HTML ---
raw_html_path = Path(config["paths"]["data_raw"]) / "goodreads_best_books.html"
save_html(soup.prettify(), raw_html_path)


## Step 3 — Parse HTML & Extract Book Information

In this step, we will parse the raw HTML file previously saved from the Goodreads list and extract all relevant book information.  
Using BeautifulSoup, we will identify and capture key fields such as:

- **Title**  
- **Author**  
- **Rating**  
- **Book link**  
- **Cover image (URL)**  
- **Genre**  
- **Price**  
- **Published year**

The extracted data will be stored in a structured pandas DataFrame, which will later be cleaned and saved in the `data/processed/` folder for further analysis and recommendation modeling.


In [ ]:
# ============================================================
# Step 3 — Parse HTML & Extract Book Information
# ============================================================

# Define the target URL for the first page
url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1"

print(f" Fetching URL: {url}")
html = fetch_html(url)

if html is None:
    print(" Could not fetch HTML. Please check the URL or your connection.")
else:
    print(" HTML fetched successfully!")

# --- Parse HTML ---
soup = parse_html(html)

# --- Extract book information into a DataFrame ---
df_page1 = extract_books_from_soup(soup)

print(f"✅ Extracted {len(df_page1)} books into DataFrame")
df_page1.head()


### Step 3.1 — Scrape Multiple Pages (~500 books)


In [ ]:
# ============================================================
# Step 3.1 — Scrape Multiple Pages from Goodreads
# ============================================================

print("🔹 Starting multi-page scraping...")

all_books = []
base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="

for page in range(1, 6):  # scrape first 5 pages ≈ 500 books
    print(f"🔹 Scraping page {page}...")
    url = base_url + str(page)
    html = fetch_html(url)
    
    if html is None:
        print(f"⚠️ Skipping page {page}, empty response.")
        continue

    soup = parse_html(html)
    books_on_page = extract_books_from_soup(soup)

    # ⬇️ CHANGED: append DataFrame instead of extend
    all_books.append(books_on_page)

    # ethical delay
    sleep(random.uniform(1, 2))

print(f"✅ Scraping completed for {len(all_books)} pages.")

# Combine all DataFrames
import pandas as pd
df = pd.concat(all_books, ignore_index=True)
print(f"✅ Combined dataset shape: {df.shape}")
df.head()


### Step 3.2 — Enrich Goodreads Data with Google Books API

In this step, we will use the **Google Books API** to fill in missing metadata for our 500 scraped books.  
For each book title and author, we will request additional information such as:
- Published year  
- Genre (category)  
- Cover image URL  

This will enrich our dataset before the cleaning and modeling steps.


In [ ]:
# ============================================================
# Step 3.2 — Enrich Goodreads Data with Google Books API
# ============================================================

import requests
from tqdm import tqdm  # progress bar

def get_book_info_from_google(title, author):
    """Query Google Books API and return metadata for a given title + author."""
    query = f"intitle:{title}+inauthor:{author}"
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if "items" in data and len(data["items"]) > 0:
                info = data["items"][0]["volumeInfo"]
                return {
                    "published_year": info.get("publishedDate", None),
                    "genre": ", ".join(info.get("categories", [])) if info.get("categories") else None,
                    "cover_url": info.get("imageLinks", {}).get("thumbnail", None)
                }
    except Exception as e:
        print(f"⚠️ Error fetching '{title}': {e}")
    
    return {"published_year": None, "genre": None, "cover_url": None}


# --- Apply API to all rows ---
print("🔹 Enriching dataset with Google Books API...")

results = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    meta = get_book_info_from_google(row["title"], row["author"])
    results.append(meta)

# Convert to DataFrame and merge
api_df = pd.DataFrame(results)
df_enriched = pd.concat([df.reset_index(drop=True), api_df], axis=1)

print(f"✅ Enrichment completed! Final shape: {df_enriched.shape}")
df_enriched.head()


## Step 3.3 — Enrich Goodreads Data with Price Information (Google Books API)

In this step, we will use the Google Books API again to retrieve price information 
(`listPrice` or `retailPrice`) for each book when available.  
This field is not available on Goodreads, so this enrichment completes our dataset 
with commercial metadata.


In [ ]:
# ============================================================
# Step 3.3 — Enrich Goodreads Data with Price Information (Google Books API)
# ============================================================

import time
def get_price_from_google(title, author):
    """Query Google Books API for price info (listPrice or retailPrice)."""
    query = f"intitle:{title}+inauthor:{author}"
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if "items" in data:
                info = data["items"][0].get("saleInfo", {})
                price_info = info.get("listPrice", {}) or info.get("retailPrice", {})
                if price_info:
                    return price_info.get("amount"), price_info.get("currencyCode")
    except Exception as e:
        print(f"⚠️ Error fetching '{title}': {e}")
    return None, None


# --- Apply to dataset ---
prices = []
for _, row in tqdm(df_enriched.iterrows(), total=len(df_enriched)):
    price, currency = get_price_from_google(row["title"], row["author"])
    prices.append({"price": price, "currency": currency})
    time.sleep(1.5)  # ethical delay

# --- Combine with original DataFrame ---
df_prices = pd.DataFrame(prices)
df_enriched_prices = pd.concat([df_enriched.reset_index(drop=True), df_prices], axis=1)

print(f"✅ Price enrichment completed: {len(df_enriched_prices)} books processed.")
df_enriched_prices.head()


# Step 4 — Final Cleaning & Save Dataset (with prices)

In this step, we perform the final data cleaning and consolidation of all book information
gathered from both sources:

- **Step 3.2:** Google Books API enrichment (adds `genre`, `cover_url`, `published_year`)
- **Step 3.3:** Price API enrichment (adds `price`, `currency`)

The goal is to standardize all fields, remove duplicates, and ensure the dataset
is ready for exploratory analysis and model training.

We will:
- Normalize text columns (`title`, `author`, `genre`, `currency`)
- Extract only the numeric rating values
- Parse and extract the publication year from full dates
- Convert `price` to numeric values
- Remove duplicates and missing titles/authors
- Keep only the relevant columns for analysis
- Save the final dataset as `books_clean.csv` inside the `data/clean/` folder

This cleaned dataset will be used for EDA (Step 5) and later integrated with additional
data from the Open Library API to build a complete Book Recommendation System.


In [ ]:
# ============================================================
# Step 4 — Final Cleaning & Save Dataset (with prices)
# ============================================================
from pathlib import Path
import re

# --- Ensure the right DataFrame is loaded ---
# If the kernel was restarted, reload from backup or enriched variable
if "df_enriched_prices" not in locals():
    clean_folder = Path(config["paths"]["data_clean"])
    enriched_path = clean_folder / "books_enriched_with_prices.csv"

    if enriched_path.exists():
        df_enriched_prices = pd.read_csv(enriched_path)
        print(f"🔄 Reloaded enriched data from: {enriched_path}")
    else:
        raise FileNotFoundError("❌ Missing df_enriched_prices in memory and CSV not found. Please re-run Step 3.3 once.")

# --- Load or copy from the enriched dataset (3.2 + 3.3 combined) ---
df_clean = df_enriched_prices.copy()
df_clean = df_clean.loc[:, ~df_clean.columns.duplicated(keep="last")]

# --- Clean text fields safely ---
for col in ["title", "author", "genre", "currency"]:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).apply(
            lambda x: x.strip() if isinstance(x, str) else x
        )

# --- Extract avg_rating as numeric ---
if "rating" in df_clean.columns:
    df_clean["avg_rating"] = (
        df_clean["rating"].astype(str).str.extract(r"(\d+\.\d+)")[0].astype(float)
    )

# --- Extract only year from published_year ---
def extract_year(date_str):
    if pd.isna(date_str):
        return None
    match = re.match(r"(\d{4})", str(date_str))
    return int(match.group(1)) if match else None

if "published_year" in df_clean.columns:
    df_clean["published_year"] = df_clean["published_year"].apply(extract_year)

# --- Convert price column to numeric ---
if "price" in df_clean.columns:
    df_clean["price"] = pd.to_numeric(df_clean["price"], errors="coerce")

# --- Drop duplicates and missing titles/authors ---
df_clean = df_clean.drop_duplicates(subset=["title"]).dropna(subset=["title", "author"])

# --- Keep only relevant columns ---
keep_cols = [
    "title",
    "author",
    "avg_rating",
    "genre",
    "published_year",
    "price",
    "currency",
    "cover_url",
    "link",
]
df_clean = df_clean[[col for col in keep_cols if col in df_clean.columns]]

# --- Save cleaned dataset ---
output_path = Path(config["paths"]["data_clean"]) / "books_clean.csv"
save_dataset(df_clean, output_path)

# ✅ Also save a backup
backup_path = Path(config["paths"]["data_clean"]) / "books_clean_backup.csv"
df_clean.to_csv(backup_path, index=False, encoding="utf-8-sig")

print(f"✅ Final cleaned dataset saved successfully: {output_path}")
print(f"💾 Backup saved at: {backup_path}")
print(f"Rows: {len(df_clean)}, Columns: {len(df_clean.columns)}")

# --- Preview ---
df_clean.head()


## 📈 Step 5 — Exploratory Data Analysis (EDA)

In this step, we performed an initial exploratory analysis of the cleaned dataset to understand its structure, completeness, and key characteristics.  
We examined rating distributions, author frequency, and potential missing values, which will guide the next steps in data enrichment and modeling.


In [ ]:
# ============================================================
# Step 5 — Exploratory Data Analysis (EDA)
# ============================================================

import importlib, functions
importlib.reload(functions)
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# --- Load cleaned dataset ---
clean_path = Path(config["paths"]["data_clean"]) / "books_clean.csv"
df = pd.read_csv(clean_path)
print(f"✅ Dataset loaded: {clean_path}")
print(df.shape)
print(df.columns.tolist())

# --- Quick overview ---
display(df.head())
display(df.describe(include='all'))

# --- Check for missing values ---
print("\n🔍 Missing values per column:")
print(df.isna().sum())

# ============================================================
# 🔎 Quick Summary before Visual Analysis
# ============================================================

total_books = len(df)
unique_authors = df["author"].nunique()
avg_rating_mean = df["avg_rating"].mean()
top_genre = df["genre"].mode()[0] if not df["genre"].mode().empty else "N/A"
most_common_year = int(df["published_year"].mode()[0]) if not df["published_year"].mode().empty else "N/A"

# --- Price metrics (new) ---
books_with_price = df["price"].notna().sum() if "price" in df.columns else 0
avg_price = df["price"].mean() if "price" in df.columns else None
currency_mode = df["currency"].mode()[0] if "currency" in df.columns and not df["currency"].mode().empty else "N/A"

print("\n📘 --- Quick Summary ---")
print(f"Total books in dataset: {total_books}")
print(f"Unique authors: {unique_authors}")
print(f"Average book rating: {avg_rating_mean:.2f}")
print(f"Most common genre: {top_genre}")
print(f"Most common publication year: {most_common_year}")
print(f"Books with price info: {books_with_price} ({books_with_price/total_books*100:.1f}%)")
if avg_price:
    print(f"Average price: {avg_price:.2f} {currency_mode}")

# ============================================================
# 📊 EDA Visualizations
# ============================================================

sns.set(style="whitegrid", palette="crest")

# --- Plot 1: Distribution of average ratings ---
plt.figure(figsize=(8, 5))
sns.histplot(df["avg_rating"], bins=15, kde=True)
plt.title("Distribution of Average Book Ratings", fontsize=13)
plt.xlabel("Average Rating")
plt.ylabel("Number of Books")
plt.show()

# --- Plot 2: Top 10 Authors by Number of Books ---
top_authors = (
    df["author"]
    .value_counts()
    .head(10)
    .sort_values(ascending=True)
)

plt.figure(figsize=(8, 5))
top_authors.plot(kind="barh", color="mediumseagreen")
plt.title("Top 10 Authors by Number of Books", fontsize=13)
plt.xlabel("Number of Books")
plt.ylabel("Author")
plt.show()

# --- Plot 3: Books by Published Year ---
plt.figure(figsize=(8, 5))
sns.countplot(
    y="published_year",
    data=df,
    hue="published_year",
    legend=False,
    dodge=False,
    order=df["published_year"].value_counts().index[:15],
    palette="viridis"
)
plt.title("Books by Published Year", fontsize=13)
plt.xlabel("Count")
plt.ylabel("Year")
plt.show()

# --- Plot 4: Top 10 Genres ---
top_genres = (
    df["genre"]
    .value_counts()
    .head(10)
    .sort_values(ascending=True)
)

plt.figure(figsize=(8, 5))
top_genres.plot(kind="barh", color="cornflowerblue")
plt.title("Top 10 Genres", fontsize=13)
plt.xlabel("Number of Books")
plt.ylabel("Genre")
plt.show()

# --- Plot 5: Price Distribution (new) ---
if "price" in df.columns and df["price"].notna().sum() > 0:
    plt.figure(figsize=(8, 5))
    sns.histplot(df["price"], bins=20, kde=True, color="darkorange")
    plt.title("Distribution of Book Prices", fontsize=13)
    plt.xlabel("Price")
    plt.ylabel("Number of Books")
    plt.show()

# --- Plot 6: Average Price by Genre (new) ---
if "price" in df.columns and "genre" in df.columns:
    genre_prices = df.groupby("genre")["price"].mean().sort_values(ascending=True).head(10)
    plt.figure(figsize=(8, 5))
    genre_prices.plot(kind="barh", color="goldenrod")
    plt.title("Average Price by Genre", fontsize=13)
    plt.xlabel("Average Price")
    plt.ylabel("Genre")
    plt.show()


### ✅ Step 5 Summary

The dataset contains **493 books** from **330 unique authors**, primarily in the *Fiction* genre.  
Average ratings hover around **4.1**, indicating that most books are highly rated and well-reviewed by readers.

Roughly **46% of the books include price information**, with an average price of **9.55 EUR** —  
a typical range for e-books or paperback editions. Prices are right-skewed, meaning that  
most books are affordable, while only a few premium titles are priced above 40 EUR.

Only a few missing values remain in non-critical columns (`genre`, `cover_url`, `price`),  
which will not significantly impact the recommendation model.

This cleaned and analyzed dataset is now ready to be used in the next phase:  
building a **content-based recommendation system** using **TF-IDF vectorization** and **cosine similarity**.

